In [1]:
!pip install -q transformers datasets peft accelerate


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="../data/finetune/fine_tune_data.jsonl", split="train")
print("Sample training pair:")
dataset[0]


c:\Users\aisha\OneDrive\Desktop\AllyIn\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sample training pair:


{'prompt': 'What emissions violations were reported in Q2?',
 'completion': '4 new emissions events were flagged in Q2 facilities.'}

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "tiiuae/falcon-rw-1b"  # Or opt-125m for faster tests

# Load tokenizer and fix padding issue
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token A # ✅ Fix for models with no pad_token

# Load model
model = AutoModelForCausalLM.from_pretrained(model_name)
model.config.pad_token_id = tokenizer.pad_token_id  # ✅ Needed for Trainer


In [6]:
def format_prompt(example):
    return f"### Question:\n{example['prompt']}\n\n### Answer:\n{example['completion']}"

def tokenize(example):
    prompt = format_prompt(example)
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(tokenize)


Map: 100%|██████████| 10/10 [00:00<00:00, 39.10 examples/s]


In [7]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],  # For Falcon model
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 1,572,864 || all params: 1,313,198,080 || trainable%: 0.1198


In [8]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="models/lora_adapter",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [9]:
trainer.train()

c:\Users\aisha\OneDrive\Desktop\AllyIn\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


c:\Users\aisha\OneDrive\Desktop\AllyIn\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=6, training_loss=3.652273178100586, metrics={'train_runtime': 149.049, 'train_samples_per_second': 0.134, 'train_steps_per_second': 0.04, 'total_flos': 37176590991360.0, 'train_loss': 3.652273178100586, 'epoch': 2.0})

In [10]:
model.save_pretrained("models/lora_adapter")
print("✅ LoRA adapter saved to models/lora_adapter")

✅ LoRA adapter saved to models/lora_adapter
